In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lrcp").getOrCreate()
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/customer_churn.csv",
                                   header=True, inferSchema=True)
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [0]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [0]:
data.columns

Out[9]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler( inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol="features" )

output = assembler.transform(data)

In [0]:
final_data = output.select("features","churn")

train_churn, test_churn = final_data.randomSplit([0.7,0.3])

from pyspark.ml.classification import LogisticRegression

lr_churn = LogisticRegression( labelCol="churn" )

fitted_churn = lr_churn.fit(train_churn)

train_sum = fitted_churn.summary

train_sum.predictions.describe().show()

+-------+------------------+-------------------+
summary| churn| prediction|
+-------+------------------+-------------------+
 count| 626| 626|
 mean|0.1757188498402556|0.13578274760383385|
 stddev|0.3808851302271002|0.34283165887338735|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+-------------------+

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator  
preds_and_labels = fitted_churn.evaluate(test_churn)
preds_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,1....| 0|[4.66131009853411...|[0.99063447380285...| 0.0|
[27.0,8628.8,1.0,...| 0|[5.42283990805618...|[0.99560481117038...| 0.0|
[28.0,8670.98,0.0...| 0|[7.76986408615425...|[0.99957790758190...| 0.0|
[28.0,11128.95,1....| 0|[4.18581441679408...|[0.98501805876740...| 0.0|
[28.0,11204.23,0....| 0|[1.92700359089514...|[0.87291739162933...| 0.0|
[29.0,9378.24,0.0...| 0|[4.78415740475756...|[0.99170816356047...| 0.0|
[29.0,10203.18,1....| 0|[3.76295492507443...|[0.97731167026867...| 0.0|
[29.0,13240.01,1....| 0|[6.72296185553783...|[0.99879847548875...| 0.0|
[30.0,7960.64,1.0...| 1|[3.12396753194461...|[0.95787062705455...| 0.0|
[30.0,8677.28,1.0...| 0|[4.10816923887002...|[0.98382799213422...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.40538625086597...|[0.96787244553151...| 0.0|
[31.0,5387.75,0.0...| 0|[2.54626963581299...|[0.92732250538402...| 0.0|
[31.0,7073.61,0.0...| 0|[3.10054986305172...|[0.95691542067333...| 0.0|
[31.0,8829.83,1.0...| 0|[4.40225145833955...|[0.98789851088536...| 0.0|
[31.0,9574.89,0.0...| 0|[3.30716647672518...|[0.96467384641078...| 0.0|
[31.0,10058.87,1....| 0|[4.42417767537282...|[0.98815785496147...| 0.0|
[31.0,11297.57,1....| 1|[0.99364132661091...|[0.72980655227097...| 0.0|
[32.0,7896.65,0.0...| 0|[3.42145500226414...|[0.96836837028183...| 0.0|
[32.0,9036.27,0.0...| 0|[-0.1892008322234...|[0.45284038895569...| 1.0|
[32.0,9472.72,1.0...| 0|[3.89863312820716...|[0.98013309571102...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
churn_eval = BinaryClassificationEvaluator( rawPredictionCol="prediction",labelCol="churn" )

auc = churn_eval.evaluate( preds_and_labels.predictions )
auc

Out[16]: 0.7454059829059828

In [0]:
dfx = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/new_customers.csv",
                                   header=True,
                                   inferSchema=True)

In [0]:
final_lr_model = lr_churn.fit(final_data)



In [0]:
dfx.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [0]:
test_new_customers = assembler.transform(dfx)

In [0]:
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [0]:
final_results = final_lr_model.transform(test_new_customers)

final_results.select("Company","prediction").show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+